# Import

In [1]:
%matplotlib widget

In [ ]:
import os
import pickle5 as pickle
import copy

import pandas as pd
import seaborn as sns
import numpy
import torch
import scipy
import scipy.stats

import pyro
import pyro.infer
import pyro.infer.mcmc
import pyro.distributions as dist
import torch.distributions.constraints as constraints
from tqdm.auto import tqdm

from pprint import pprint

import matplotlib.pyplot as plot
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
import warnings
warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

In [ ]:
repo = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

# Define

In [ ]:
def sigmoid(x):
    return 1./(1.+torch.exp(-x))

def icc_best_deriv(alpha, beta, theta, model_names, gamma=None, col='mean', target=None):
    '''
    Method to calculate the locally estimated headroom (LEH) score, defined as
    the derivative of the item characteristic curve w.r.t. the best performing model.
    
    Args:
        alpha:       DataFrame of discrimination parameter statistics for each item.
        beta:        DataFrame of difficulty parameter statistics for each item.
        theta:       DataFrame of ability parameter statistics for each responder.
        model_names: List of responder names.
        gamma:       DataFrame of guessing parameter statistics for each item.
        col:         DataFrame column name to use for calculating LEH scores.
    
    Returns:
        scores:      LEH scores for each item.    
    '''
    if target:
        idx = model_names.index(target)
        best_value = theta[col].iloc[idx]
        print('Local Grad for Target Model')
        print(f'Target model: {target}\n{best_value}')
    else:
        best_idx, best_value = theta[col].argmax(), theta[col].max()
        print(f'Best model: {model_names[best_idx]}\n{best_value}')
    
    a, b = torch.tensor(alpha[col].values), torch.tensor(beta[col].values)
    
    logits = (a*(best_value-b))
    sigmoids = sigmoid(logits)
    scores = sigmoids*(1.-sigmoids)*a
    
    print(f'No gamma: {scores.mean()}')
    if not gamma is None:
        g = torch.tensor(gamma[col].apply(lambda x: x.item()).values)
        scores = (1.-g)*scores
        print(f'With gamma: {scores.mean()}')
    
    return scores      
    
    

In [ ]:
def get_model_guide(alpha_dist, theta_dist, alpha_transform, theta_transform):
    model = lambda obs: irt_model(obs, alpha_dist, theta_dist, alpha_transform = alpha_transform, theta_transform = theta_transform)
    guide = lambda obs: vi_posterior(obs, alpha_dist, theta_dist)
    
    return model, guide

In [ ]:
def get_data_accuracies(data, verbose = False, get_cols = False):
    '''
    Method to reformat `data` and calculate item and responder accuracies.
    
    Args:
        data:                DataFrame of item responses.
        verbose:             Boolean value of whether to print statements.
        get_cols:            Boolean value of whether to return original column
                             values of `data`.
        
    Returns:
        new_data:            Reformatted `data`, dropping first column.
        accuracies:          Accuracy for each responder across examples.
        example_accuracies:  Accuracy for each example across responders.
        data.columns.values: Returns only if `get_cols` is True. Original column
                             values of `data`.
    '''
    new_data = numpy.array(data)
    new_data = new_data[:,1:]
    
    model_names = dict(data['userid'])
    accuracies = new_data.mean(-1)
    example_accuracies = new_data.mean(0)
    
    if verbose:
        print('\n'.join([f'{name}: {acc}' for name, acc in zip(model_names.values(),accuracies)]))
    
    if get_cols:
        return new_data, accuracies, example_accuracies, data.columns.values
    else:
        return new_data, accuracies, example_accuracies

In [ ]:
def get_stats_CI(params, p=0.95, dist='normal'):
    '''
    Method to calculate lower and upper quantiles defined by `p`, mean, and variance of `param`
    
    Args:
        params: Dictionary of distribution parameters for each item keyed according to the 
                parametric distribution defined by `dist`.
        p:      Percent of distribution covered by the lower and upper interval values for each
                parameter.
        dist:   Name of parametric distribution
    
    Returns:
        return: {
            'lower': Lower interval values of each parameter,
            'upper': Upper interval values of each parameter,
            'mean' : Mean of each parameter,
            'var'  : Variance of each parameter
        }
    '''
    stats = {}
    if dist == 'normal':
        L,U = scipy.stats.norm.interval(p,loc=params['mu'], scale=torch.exp(params['logstd']))
        M,V = scipy.stats.norm.stats(loc=params['mu'], scale=torch.exp(params['logstd']))
    elif dist == 'log-normal':
        L,U = scipy.stats.lognorm.interval(p, s=torch.exp(params['logstd']), scale=torch.exp(params['mu']))
        M,V = scipy.stats.lognorm.stats(s=torch.exp(params['logstd']), scale=torch.exp(params['mu']))
    elif dist == 'beta':
        L,U = scipy.stats.beta.interval(p,a=params['alpha'], b=params['beta'])
        M,V = scipy.stats.beta.stats(a=params['alpha'], b=params['beta'])
    else:
        raise TypeError(f'Distribution type {dist} not supported.')
    
    return {
        'lower':[L],
        'upper':[U],
        'mean':[M],
        'var':[V],
    }

In [ ]:
def get_plot_stats(exp_dir, alpha_dist, theta_dist, transforms, p = 0.95):
    '''
    Method to return plotting statistics for 3 parameter IRT model parameters.
    
    Args:
        exp_dir:          Path to 3 parameter IRT parameters and responses.
        alpha_dist:       Name of the item discrimination [a] distribution.
        theta_dist:       Name of the responder ability [t] distribution.
        transforms:       Dictionary of transformations to apply to each parameter type
                          where keys are parameter names and values are functions.
        p:                Percent of distribution covered by the lower and upper interval 
                          values for each parameter.
    
    Returns:
        param_plot_stats: Dictionary of parameter plot statistics where keys are parameter
                          names and values are plot statistics dictionaries as defined by
                          get_stats_CI().
    '''
    param_dists = {
        'a':alpha_dist,
        'b':'normal',
        'g':'normal',
        't':theta_dist,
    }

    dist_params = {
        'normal':['mu', 'logstd'],
        'log-normal':['mu', 'logstd'],
        'beta':['alpha', 'beta'],
    }

    pyro.clear_param_store()
    pyro.get_param_store().load(os.path.join(exp_dir, 'params.p'))

    with torch.no_grad():
        pyro_param_dict = dict(pyro.get_param_store().named_parameters())
    
    # get stats for plotting
    param_plot_stats = {}

    for param, param_dist in param_dists.items():
        temp_params = dist_params[param_dist]

        for idx, (p1_orig, p2_orig) in enumerate(zip(pyro_param_dict[f'{param} {temp_params[0]}'], pyro_param_dict[f'{param} {temp_params[1]}'])):
            p1, p2 = p1_orig.detach(), p2_orig.detach()
            
            temp_stats_df = pd.DataFrame.from_dict(
                get_stats_CI(
                    params = {
                        temp_params[0]:p1,
                        temp_params[1]:p2,
                    },
                    p=p,
                    dist = param_dist,
                )
            )
            
            temp_stats_df = temp_stats_df.applymap(transforms[param])
        
            if idx == 0:
                param_plot_stats[param] = temp_stats_df
            else:
                param_plot_stats[param] = param_plot_stats[param].append(temp_stats_df, ignore_index = True)
    
    return param_plot_stats

In [ ]:
def sign_mult(df1, df2):
    newdf = copy.deepcopy(df2)
    
    for idx, row in df1.iterrows():
        if numpy.sign(row['mean']) < 0:
            newdf.loc[idx,'mean'] = -1*newdf.loc[idx,'mean']
            newdf.loc[idx,'lower'] = -1*newdf.loc[idx,'upper']
            newdf.loc[idx,'upper'] = -1*newdf.loc[idx,'lower']
    
    return newdf

In [ ]:
def get_diff_by_set(diffs, item_ids):
    diff_by_set = {}
    id_split = '_'

    max_diff = -1e6
    min_diff = 1e6
    
    for idx, diff in enumerate(diffs):
        set_name = item_ids[idx].split(id_split)[0]

        if set_name in diff_by_set.keys():
            diff_by_set[set_name].append(diff)
        else:
            diff_by_set[set_name] = [diff]
            
        if diff < min_diff:
            min_diff = diff
            
        if diff > max_diff:
            max_diff = diff
    
    return diff_by_set, min_diff, max_diff

# Get Tasks

In [ ]:
from variational_irt import *

In [ ]:
datasets="boolq,cb,commonsenseqa,copa,cosmosqa,hellaswag,adversarial-nli,rte,snli,wic,qamr,arct,mcscript,mctaco,mutual,mutual-plus,quoref,socialiqa,squad-v2,wsc,mnli,mrqa-nq,newsqa,abductive-nli,arc-easy,arc-challenge,piqa,quail,winogrande,anli"
data_names, responses, n_items = get_files(
    os.path.join(repo, 'data'),
    "csv",
    set(datasets.split(','))
)

In [ ]:
task_metadata = pd.read_csv('task_metadata.csv')
task_metadata.set_index("jiant_name", inplace=True)
task_list = [x for x in task_metadata.index if x in data_names]

In [ ]:
total = 0
task_name = []
task_format = []

for tname, size in zip(data_names, n_items):
    name = task_metadata.loc[tname]['taskname']
    total += size
    task_name += [name for _ in range(size)]
    task_format += [task_metadata.loc[tname]['format'] for _ in range(size)]
    
task_name = pd.DataFrame(task_name, columns=['task_name'])
task_format = pd.DataFrame(task_format, columns=['format'])
task_name_format = pd.concat([task_name, task_format], axis=1)

In [ ]:
len(data_names)

# Get Params and Order

In [ ]:
exp_dir = os.path.join(repo, 'params', f'alpha-lognormal-identity_theta-normal-identity_nosubsample_1.00_0.30')
p = 0.95

with open(os.path.join(exp_dir, 'responses.p'), 'rb') as f:
    combined_responses = pickle.load(f).reset_index()

In [ ]:
# Check accuracy of roberta-large models

extractmodel = 'roberta-large_best'
tie_break = 0

acc_by_dataset = {}

roberta_rp = combined_responses.loc[combined_responses['userid']==extractmodel, :]
if roberta_rp.shape[0] > 1:
    roberta_rp = roberta_rp.iloc[tie_break, :]

cols = combined_responses.columns.values

for item in cols[1:]:
    data_name = '_'.join(item.split('_')[:-1])
    resp = roberta_rp[item].item()
    
    if data_name in acc_by_dataset:
        acc_by_dataset[data_name]['correct'] += resp
        acc_by_dataset[data_name]['total'] += 1
    else:
        acc_by_dataset[data_name] = {'correct': resp, 'total': 1}

print(extractmodel)
print('='*90)
print(f'Overall acc: {roberta_rp.iloc[0, 1:].sum()/(roberta_rp.shape[1]-1):.4f}')        

for data_name, acc_dict in acc_by_dataset.items():
    print(f'{data_name} acc: {acc_dict["correct"]/acc_dict["total"]:.4f}')

In [ ]:
# set to False if run for the first time
# note that this will take sometimes to run if the datasets are big
load_from_cache = True

In [ ]:
# distribution and transformation
alpha_dist = 'log-normal'
alpha_transf = 'standard'
theta_dist = 'normal'
theta_transf = 'standard'

exp_dir = os.path.join(repo, 'params', f'alpha-lognormal-identity_theta-normal-identity_nosubsample_1.00_0.30')
p = 0.95

with open(os.path.join(exp_dir, 'responses.p'), 'rb') as f:
    combined_responses = pickle.load(f).reset_index()
data, accuracies, example_accuracies = get_data_accuracies(combined_responses)
column_names = combined_responses.columns[1:]
select_ts = {
    'standard':lambda x:x,
    'positive':lambda x:torch.log(1+torch.exp(torch.tensor(x))),
    'sigmoid':lambda x:sigmoid(torch.tensor(x)),
}

transforms = {
    'a':select_ts[alpha_transf],
    'b':select_ts['standard'],
    'g':select_ts['sigmoid'],
    't':select_ts[theta_transf],
}

if load_from_cache:
    param_plot_stats = {}

    for key in transforms.keys():
        with open(os.path.join('plot_stats_pickles', f'{key}.p'), 'rb') as f:
            param_plot_stats[key] = pickle.load(f)
else:
    param_plot_stats = get_plot_stats(
        exp_dir,
        alpha_dist,
        theta_dist,
        transforms,
        p = 0.95
    )
    
    os.makedirs('plot_stats_pickles', exist_ok=True)
    for key, value in param_plot_stats.items():
        with open(os.path.join('plot_stats_pickles', f'{key}.p'), 'wb') as f:
            pickle.dump(value, f)

In [ ]:
combined_responses

In [ ]:
model_names = []
model_levels = []
for m in combined_responses['userid']:
    mname = m.split('_')[0]
    mlevel = m.split('_')[-1]
    if mname.endswith('-1') or mname.endswith('-2') or mname.endswith('-3'):
        mname = mname[:-2]
    model_names.append(mname)
    
    mlevel_append = '' if mlevel == 'best' else r'%'
    model_levels.append(mlevel+mlevel_append)

In [ ]:
#  we will only use log mean for discriminative parameter
for param_key, param_stat in param_plot_stats.items():
    param_stat['log_mean'] = numpy.log(param_stat['mean'])
    print(param_key, param_stat['log_mean'].isnull().sum())

In [ ]:
param_a = pd.concat([param_plot_stats['a'], task_name_format], axis=1)
param_b = pd.concat([param_plot_stats['b'], task_name_format], axis=1)

task_order = [task_metadata.loc[x]['taskname'] for x in task_list]

In [ ]:
leh_scores = icc_best_deriv(
    param_plot_stats['a'],
    param_plot_stats['b'],
    param_plot_stats['t'],
    model_names,
    gamma = param_plot_stats['g'],
)

leh_scores_plot = pd.DataFrame(pd.Series(leh_scores), columns = ['mean'])
print(leh_scores_plot)

In [ ]:
leh_scores_plot = pd.concat([leh_scores_plot, task_name_format], axis=1)

In [ ]:
leh_75 = leh_scores_plot.groupby(by='task_name').quantile(q=0.75).reset_index()

with open(os.path.join('plot_stats_pickles', f'LEH_75qtile.p'), 'wb') as f:
    pickle.dump(leh_75, f)

In [ ]:
task_metadata.set_index("taskname", inplace=True)

# Follow-ups

## % all incorrect/correct

In [ ]:
pcent_correct = pd.DataFrame(
    combined_responses.iloc[:,1:].sum(axis=0)/combined_responses.shape[0], columns=['pcent_correct']
)

In [ ]:
pcent_correct['all_correct'] = pcent_correct['pcent_correct'].eq(1)
pcent_correct['all_wrong'] = pcent_correct['pcent_correct'].eq(0)
pcent_correct['all_either'] = pcent_correct['all_correct'] + pcent_correct['all_wrong']
pcent_correct['dataset'] = list(pd.Series(pcent_correct.index.values).apply(lambda x: '_'.join(x.split('_')[:-1])))

In [ ]:
pcent_correct

In [ ]:
print('exclude', param_plot_stats['a'].loc[~np.array(pcent_correct['all_either']),:].mean())
print('')
print('only', param_plot_stats['a'].loc[np.array(pcent_correct['all_either']),:].mean())
print('')
print('only correct', param_plot_stats['a'].loc[np.array(pcent_correct['all_correct']),:].mean())
print('')
print('only wrong', param_plot_stats['a'].loc[np.array(pcent_correct['all_wrong']),:].mean())

In [ ]:
print('exclude', param_plot_stats['g'].loc[~np.array(pcent_correct['all_either']),:].mean())
print('')
print('only', param_plot_stats['g'].loc[np.array(pcent_correct['all_either']),:].mean())
print('')
print('only correct', param_plot_stats['g'].loc[np.array(pcent_correct['all_correct']),:].mean())
print('')
print('only wrong', param_plot_stats['g'].loc[np.array(pcent_correct['all_wrong']),:].mean())

In [ ]:
temp = pd.concat([pcent_correct.reset_index(), task_name_format], axis=1)

dataset2format = {'all':'all'}
format2dataset = {'all':'all'}

for dset in temp['dataset'].unique():
    fmat = temp.loc[temp['dataset'] == dset, 'format'].unique()[0]
    dataset2format[dset] = fmat
    format2dataset[fmat] = dset

In [ ]:
item_excludes = {}

for dataset in pcent_correct['dataset'].unique():
    temp = pcent_correct.loc[pcent_correct['dataset'] == dataset, :]
    
    try:
        assert np.not_equal(temp.index.values, np.array(sorted(temp.index.values, key=lambda x: int(x.split('_')[-1])))).sum() == 0, dataset
    except:
        print(dataset)
        pass
    
    item_excludes[dataset] = temp['all_either']

In [ ]:
keeps = ['all_correct', 'all_wrong', 'all_either', 'dataset']
grouped_pcent_correct = pcent_correct[keeps].groupby(by='dataset')

In [ ]:
combined_all_pcent = {}
combined_all_count = {}

for col in ['all_correct', 'all_wrong', 'all_either']:
    combined_all_count[col] = pcent_correct[col].sum()
    combined_all_pcent[col] = combined_all_count[col]/combined_responses.shape[1]
    
combined_all_count['total_count'] = combined_responses.shape[1]

In [ ]:
grouped_pcent_either = grouped_pcent_correct.sum()/grouped_pcent_correct.count()
summary_pcent = grouped_pcent_either.append(pd.DataFrame.from_dict({'all':combined_all_pcent}, orient='index'))

grouped_count_either = grouped_pcent_correct.sum()
grouped_count_either['total_count'] = grouped_pcent_correct.count()['all_either']
summary_count = grouped_count_either.append(pd.DataFrame.from_dict({'all':combined_all_count}, orient='index'))


summary_pcent['format'] = list(pd.Series(summary_pcent.index.values).apply(lambda x: dataset2format[x]))
summary_count['format'] = list(pd.Series(summary_count.index.values).apply(lambda x: dataset2format[x]))

In [ ]:
summary_pcent.loc[summary_pcent['all_either'] > 0, :]

In [ ]:
summary_count.loc[summary_count['all_either'] > 0, :]

In [ ]:
summary_by_format = summary_count.groupby(by='format')
summary_by_format = summary_by_format.sum()

In [ ]:
summary_by_format.loc[summary_by_format['all_either'] > 0,:]

In [ ]:
print('Break Down of All Either')
summary_by_format['all_either']/summary_by_format.loc['all', 'all_either']

In [ ]:
print('Breakdown of All Either by Task Format')
summary_by_format['all_either']/summary_by_format['total_count']

## Trim Responses

In [ ]:
data_dir = os.path.join('..','data')
files = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]

### Trim Model Responses

In [ ]:
trimmed_model_dir = os.path.join('..','data_trimmed_model')
os.makedirs(trimmed_model_dir, exist_ok=True)

In [ ]:
exclude_models = ['albert-xxlarge-v2', 'xlm-roberta-large', 'roberta-large', 'roberta-base']

In [ ]:
for file in files:
    temp = pd.read_csv(os.path.join(data_dir, file))
    temp_models = temp['userid'].apply(lambda x:''.join(x.split('_')[:-1]))
    
    exclude_indexes = temp_models.eq(exclude_models[0])
    for i in range(1,len(exclude_models)):
        exclude_indexes = exclude_indexes + temp_models.eq(exclude_models[i])
    
    trimmed = temp.loc[~exclude_indexes, :]
    
    check = set(temp.loc[~exclude_indexes, :]['userid'].apply(lambda x:''.join(x.split('_')[:-1])).unique())
    for model in exclude_models:
        assert not model in check, f'{file}, {model}'
    
    trimmed.to_csv(os.path.join(trimmed_model_dir, file), index=False)

### Trim Item Responses

In [ ]:
trimmed_item_dir = os.path.join('..','data_trimmed_item')
os.makedirs(trimmed_item_dir, exist_ok=True)

In [ ]:
ending = '_irt_all_coded.csv'

for file in files:
    dataset_name = '_'.join(file[:-len(ending)].split('-'))
    if dataset_name == 'mrqa_nq':
        dataset_name = 'mrqa_natural_questions'
    
    if not dataset_name in item_excludes.keys():
        print(dataset_name)  
    
    temp = pd.read_csv(os.path.join(data_dir, file))
    trimmed = temp.loc[:,~np.array([False] + list(item_excludes[dataset_name]))]
    
    trimmed.to_csv(os.path.join(trimmed_item_dir, file), index=False)

# LEH at BERT-large

### From Total

In [ ]:
target = 'bert-large-cased_best'

leh_scores_bert = icc_best_deriv(
    param_plot_stats['a'],
    param_plot_stats['b'],
    param_plot_stats['t'],
    list(combined_responses['userid']),
    gamma = param_plot_stats['g'],
    target = target
)

leh_scores_plot_bert = pd.DataFrame(pd.Series(leh_scores_bert), columns = ['mean'])
print(leh_scores_plot_bert)

### From Trimmed BERT

In [ ]:
# set to False if run for the first time
# note that this will take sometimes to run if the datasets are big
load_from_cache_trimmed = True

In [ ]:
# distribution and transformation
alpha_dist = 'log-normal'
alpha_transf = 'standard'
theta_dist = 'normal'
theta_transf = 'standard'

exp_dir_trimmed = os.path.join(repo, 'params_trimmed_model', f'alpha-lognormal-identity_theta-normal-identity_nosubsample_1.00_0.30')
p = 0.95

with open(os.path.join(exp_dir_trimmed, 'responses.p'), 'rb') as f:
    combined_responses_trimmed = pickle.load(f).reset_index()
data_trimmed, accuracies_trimmed, example_accuracies_trimmed = get_data_accuracies(combined_responses_trimmed)
column_names_trimmed = combined_responses_trimmed.columns[1:]
select_ts = {
    'standard':lambda x:x,
    'positive':lambda x:torch.log(1+torch.exp(torch.tensor(x))),
    'sigmoid':lambda x:sigmoid(torch.tensor(x)),
}

transforms = {
    'a':select_ts[alpha_transf],
    'b':select_ts['standard'],
    'g':select_ts['sigmoid'],
    't':select_ts[theta_transf],
}

if load_from_cache_trimmed:
    param_plot_stats_trimmed = {}

    for key in transforms.keys():
        with open(os.path.join('plot_stats_pickles_trimmed_model', f'{key}.p'), 'rb') as f:
            param_plot_stats_trimmed[key] = pickle.load(f)
else:
    param_plot_stats_trimmed = get_plot_stats(
        exp_dir_trimmed,
        alpha_dist,
        theta_dist,
        transforms,
        p = p
    )
    
    os.makedirs('plot_stats_pickles_trimmed_model', exist_ok=True)
    for key, value in param_plot_stats_trimmed.items():
        with open(os.path.join('plot_stats_pickles_trimmed_model', f'{key}.p'), 'wb') as f:
            pickle.dump(value, f)

In [ ]:
target = 'bert-large-cased_best'

leh_scores_bert_trimmed = icc_best_deriv(
    param_plot_stats_trimmed['a'],
    param_plot_stats_trimmed['b'],
    param_plot_stats_trimmed['t'],
    list(combined_responses_trimmed['userid']),
    gamma = param_plot_stats_trimmed['g'],
    target = target
)

leh_scores_plot_bert_trimmed = pd.DataFrame(pd.Series(leh_scores_bert_trimmed), columns = ['mean'])
print(leh_scores_plot_bert_trimmed)

## Differences

In [ ]:
leh_bert_combined = pd.concat([leh_scores_plot_bert, leh_scores_plot_bert_trimmed], axis = 1)

In [ ]:
diff = (leh_scores_plot_bert['mean'] - leh_scores_plot_bert_trimmed['mean'])

print('mean', diff.mean())
print('std', diff.std())
print('median', diff.median())

IQR = diff.quantile(0.75) - diff.quantile(0.25)
upper_whisker = diff.quantile(0.75) + 1.5*IQR
lower_whisker = diff.quantile(0.25) - 1.5*IQR

print('IQR', IQR)
print('upper_whisker', upper_whisker)
print('lower_whisker', lower_whisker)

print('pcent above lower', (diff > upper_whisker).sum()/diff.shape[0])
print('pcent below lower', (diff < lower_whisker).sum()/diff.shape[0])

In [ ]:
fig, ax = plot.subplots()
sns.boxplot(diff)
ax.set_xlabel(r'$\Delta$ LEH')
ax.set_title('LEH at BERT: Difference')

In [ ]:
fig, ax = plot.subplots()
sns.boxplot(diff, showfliers=False)
ax.set_xlabel(r'$\Delta$ LEH')
ax.set_title('LEH at BERT: Difference')

In [ ]:
rel_diff = (leh_scores_plot_bert['mean'] - leh_scores_plot_bert_trimmed['mean'])/leh_scores_plot_bert_trimmed['mean']

print('mean', rel_diff.mean())
print('std', rel_diff.std())
print('median', rel_diff.median())

IQR = rel_diff.quantile(0.75) - rel_diff.quantile(0.25)
upper_whisker = rel_diff.quantile(0.75) + 1.5*IQR
lower_whisker = rel_diff.quantile(0.25) - 1.5*IQR

print('IQR', IQR)
print('upper_whisker', upper_whisker)
print('lower_whisker', lower_whisker)

print('pcent above lower', (rel_diff > upper_whisker).sum()/rel_diff.shape[0])
print('pcent below lower', (rel_diff < lower_whisker).sum()/rel_diff.shape[0])

In [ ]:
fig, ax = plot.subplots()
sns.boxplot(rel_diff)
ax.set_xlabel(r'$\Delta$ LEH / LEH$_{BERT}$')
ax.set_title('LEH at BERT: Relative Difference')

In [ ]:
fig, ax = plot.subplots()
sns.boxplot(rel_diff, showfliers=False)
ax.set_xlabel(r'$\Delta$ LEH / LEH$_{BERT}$')
ax.set_title('LEH at BERT: Relative Difference')

## Boxplots

### Methods

In [ ]:
font_label = 12
font_legend = 10
font_legendtitle = font_legend + 4
font_xtick = 12
font_title = 14
marker_scale = 1.5

plot.rc('axes', labelsize=font_label)
plot.rc('axes', titlesize=font_title)
plot.rc('legend', fontsize=font_legend)

In [ ]:
def plot_box(df, order, param_type, task_metadata, xsize=12, ysize=3, width=0.6, rotation=90, ylim=None, ystep=None):
    
    param2label = {
        'discriminative':r'$\log$ Discrimination ($\log$ $\alpha$)',
        'difficulty': r'Difficulty ($\beta$)',
        "disc-diff": "IRT Score", # "discrimination - difficulty product\n", # + r"($ n(\alpha) x n(\beta$) )"
        "disc-diff_pos": "IRT Score", # "discrimination - difficulty product\n", # + r"($ n(\alpha) x n(\beta$) )"
        "disc-diff_minmax": "IRT Score", # "discrimination - difficulty product\n", # + r"($ n(\alpha) x n(\beta$) )"
        "irt-score": "LEH Score",
    }
    
    param2yname = {
        'discriminative': "log_mean",
        'difficulty': "mean",
        "disc-diff": 0,
        "disc-diff_pos": 0,
        "disc-diff_minmax": 0,
        "irt-score": "mean",
    }
    
    sns.set_style("whitegrid")
    sns.set_context("paper")
    f, ax = plot.subplots(figsize=(xsize, ysize))
    
    my_pal = {"MC-par": "r",
              "MC-sent": "b",
              "classification":"g",
              "span selection": "grey"}    
    
    ax = sns.boxplot(x="task_name", y=param2yname[param_type], data=df, order=order, width=width)
    
    for i, task in enumerate(order):
        # Select which box you want to change    
        mybox = ax.artists[i]
        
        # Change the appearance of that box
        skill = task_metadata.loc[task]['format']
        mybox.set_facecolor(my_pal[skill])
    
    # Add transparency to colors
    for patch in ax.artists:
         r, g, b, a = patch.get_facecolor()
         patch.set_facecolor((r, g, b, .6))

    sns.despine()
    plot.xticks(range(len(order)), order, rotation=rotation, fontsize=font_xtick)
    
    if not ylim is None and not ystep is None:
        plot.ylim(ylim)
        plot.yticks(numpy.arange(ylim[0], ylim[1]+ystep, ystep))
    
    plot.xlabel(None)
    plot.ylabel(param2label[param_type], fontsize=font_label)
    
    return f
#     plot.savefig('../plots/' + param_type + "_box.png",
#                 format='png', dpi=300,
#                 bbox_inches = 'tight',
#                 pad_inches = .1)

### Plots

In [ ]:
leh_bert_combined.corr()

In [ ]:
leh_scores_plot_bert = pd.concat([leh_scores_plot_bert, task_name_format], axis=1)
leh_scores_plot_bert_trimmed = pd.concat([leh_scores_plot_bert_trimmed, task_name_format], axis=1)

In [ ]:
leh_scores_plot_bert

In [ ]:
f = plot_box(
    leh_scores_plot_bert,
    task_order,
    "irt-score",
    task_metadata
)

f.suptitle('All Models')
f.savefig(os.path.join('..', 'plots_LEH_BERT', 'all_models.png'),
         format='png', dpi=300,
                bbox_inches = 'tight',
                pad_inches = .1)

In [ ]:
ftrimmed = plot_box(
    leh_scores_plot_bert_trimmed,
    task_order,
    "irt-score",
    task_metadata
)

ftrimmed.suptitle('MiniBERTas + BERT')
ftrimmed.savefig(os.path.join('..', 'plots_LEH_BERT', 'trimmed_models.png'),
         format='png', dpi=300,
                bbox_inches = 'tight',
                pad_inches = .1)

In [ ]:
leh_tasks = pd.concat([
    leh_scores_plot_bert.rename(columns={'mean':'Full'}),
    pd.DataFrame(leh_scores_plot_bert_trimmed['mean']).rename(columns={'mean':'BERT'})
], axis=1)

In [ ]:
leh_tasks_qtile = leh_tasks.groupby(by='task_name').quantile(q=0.75)
leh_tasks_qtile['diff'] = leh_tasks_qtile['Full'] - leh_tasks_qtile['BERT']
leh_tasks_qtile['rel_diff'] = (leh_tasks_qtile['Full'] - leh_tasks_qtile['BERT'])/leh_tasks_qtile['BERT']

print('diff', leh_tasks_qtile['diff'].median())
print('rel diff', leh_tasks_qtile['rel_diff'].median())

In [ ]:
print('median diff (magnitude)', leh_tasks_qtile['diff'].abs().median())
print('median rel diff (magnitude)', leh_tasks_qtile['rel_diff'].abs().median())

In [ ]:
leh_tasks_qtile.rename(columns={'BERT':'Up to BERT', 'diff':'Difference', 'rel_diff':'Relative Difference'}).sort_values(by='Relative Difference')

In [ ]:
leh_tasks_qtile.loc[leh_tasks_qtile['diff'] < 0, :]